## Hardware check

In [1]:
# gpu check
! nvidia-smi

Thu May 30 14:58:15 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:3A:00.0 Off |                    0 |
| N/A   30C    P0             41W /  300W |      17MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
cores

80

In [3]:
! python --version

Python 3.9.16


## Environment Prep

In [4]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis/

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [5]:
from keras.utils import pad_sequences
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import json

2024-05-30 14:58:22.919065: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-30 14:58:23.599305: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 14:58:49.897793: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Train/Val

In [6]:
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]
# embs = pkl[5]
del pkl

In [7]:
data_path = 'Data/CLS_emb_keywords_reserved.pkl'
pkl = pickle.load(open(data_path, 'rb'))
embs = pkl[0]
del pkl

In [8]:
text_data = data[data['variable'] == 'Text']
embs_list = []

for emb in embs:
    embs_list.append(emb)

text_data['value'] = embs_list
physio_data = data[data['variable'] != 'Text']

del data
# data = text_data.append(physio_data, ignore_index=False)

/scratch/slurm_tmpdir/job_23662206/ipykernel_684549/1507988505.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data['value'] = embs_list


In [9]:
data = text_data.reset_index()
del data['index']
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,"[0.14401615, 0.15158324, -0.3973308, 0.26409, ...",noteevents,1.0,1.0
1,18407,28.016667,Text,"[0.10852944, 0.2838851, -0.3647646, 0.27892292...",noteevents,1.0,1.0
2,40300,155.166667,Text,"[0.064612664, 0.20329593, -0.4460111, 0.203105...",noteevents,1.0,1.0
3,23747,52.383333,Text,"[0.070009716, 0.33203533, -0.21765643, 0.15761...",noteevents,1.0,1.0
4,2357,73.133333,Text,"[0.1063, 0.10509527, -0.35790482, 0.10100723, ...",noteevents,1.0,1.0
...,...,...,...,...,...,...,...
1407425,31015,99.400000,Text,"[-0.06811574, 0.019226989, -0.59367394, 0.2867...",noteevents,1.0,1.0
1407426,31015,105.783333,Text,"[0.18735209, 0.23587628, -0.03283742, 0.438380...",noteevents,1.0,1.0
1407427,31015,112.650000,Text,"[-0.18193512, -0.00037389036, -0.4246634, 0.31...",noteevents,1.0,1.0
1407428,31015,114.033333,Text,"[-0.099320516, 0.38617504, -0.54376817, 0.2740...",noteevents,1.0,1.0


In [10]:
data = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
data = data.loc[(data.hour>=0)&(data.hour<=24)]
N = data.ts_ind.max() + 1
N

57282

In [11]:
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds==0)*1 + (stds!=0)*stds
demo = (demo-means)/stds
# Trim to max len.
data = data.sample(frac=1)
# data = data.groupby('ts_ind').head(880)
# Get N, V, var_to_ind.
N = data.ts_ind.max() + 1
varis = sorted(list(set(data.variable)))
V = len(varis)
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Add obs index.
data = data.sort_values(by=['ts_ind']).reset_index(drop=True)
data = data.reset_index().rename(columns={'index':'obs_ind'})
data = data.merge(data.groupby('ts_ind').agg({'obs_ind':'min'}).reset_index().rename(columns={ \
                                                            'obs_ind':'first_obs_ind'}), on='ts_ind')
data['obs_ind'] = data['obs_ind'] - data['first_obs_ind']
# Find max_len.
max_len = data.obs_ind.max()+1
print ('max_len', max_len)

0it [00:00, ?it/s]


max_len 40


In [12]:
data = data.sort_values(['ts_ind','hour'])
data

,obs_ind,ts_ind,vind,hour,value,first_obs_ind
0,0,0,1,5.683333,"[-0.20860982, 0.21185082, -0.3639829, 0.432604...",0
1,1,0,1,9.066667,"[0.093354665, -0.27403927, -0.055233687, 0.274...",0
2,2,0,1,11.133333,"[0.0120629845, -0.1610519, -0.004898468, 0.126...",0
3,3,0,1,15.666667,"[0.07136194, -0.3218898, -0.11753161, 0.273947...",0
4,4,0,1,18.933333,"[-0.07164511, 0.30317348, -0.2000821, 0.364235...",0
...,...,...,...,...,...,...
316730,2,57280,1,16.966667,"[-0.038624022, 0.12539336, -0.17425126, 0.2528...",316728
316731,0,57281,1,1.000000,"[0.014610085, 0.19689673, -0.25572744, 0.52322...",316731
316732,1,57281,1,5.400000,"[0.21962649, 0.031458337, 0.13265967, 0.087429...",316731
316733,2,57281,1,9.300000,"[-0.06340878, 0.37905407, -0.30367488, 0.21357...",316731


In [13]:
texts_inp = np.empty([N, max_len], dtype=object)
for row in tqdm(data.itertuples()):
    ts_ind = row.ts_ind
    l = row.obs_ind
    texts_inp[ts_ind, l] = row.hour

316735it [00:00, 803846.21it/s]


In [14]:
data.drop(columns=['obs_ind', 'first_obs_ind'], inplace=True)
train_ip = [ip[train_ind] for ip in [texts_inp]]
valid_ip = [ip[valid_ind] for ip in [texts_inp]]
test_ip = [ip[test_ind] for ip in [texts_inp]]
del texts_inp

In [15]:
train_text_times = train_ip[0]
train_text_times_dense = []
train_varis = []

for times in tqdm(train_text_times):
    times_cur = []
    varis_cur = []
    for time in times:
        # get rid of the None
        if time is not None:
            times_cur.append(time)
            varis_cur.append(135)
    train_text_times_dense.append(np.array(times_cur))
    train_varis.append(np.array(varis_cur))

100%|██████████| 36551/36551 [00:00<00:00, 195001.90it/s]


In [16]:
valid_text_times = valid_ip[0]
valid_text_times_dense = []
valid_varis = []

for times in tqdm(valid_text_times):
    times_cur = []
    varis_cur = []
    for time in times:
        # get rid of the None
        if time is not None:
            times_cur.append(time)
            varis_cur.append(135)
    valid_text_times_dense.append(np.array(times_cur))
    valid_varis.append(np.array(varis_cur))

100%|██████████| 9262/9262 [00:00<00:00, 183772.38it/s]


In [17]:
test_text_times = test_ip[0]
test_text_times_dense = []
test_varis = []

for times in tqdm(test_text_times):
    times_cur = []
    varis_cur = []
    for time in times:
        # get rid of the None
        if time is not None:
            times_cur.append(time)
            varis_cur.append(135)
    test_text_times_dense.append(np.array(times_cur))
    test_varis.append(np.array(varis_cur))

100%|██████████| 11469/11469 [00:00<00:00, 189048.29it/s]


In [18]:
# padding
max_len = 50
# padding
padded_train = pad_sequences(train_text_times_dense, maxlen=max_len, dtype='float32', padding='post')
padded_train.shape

(36551, 50)

In [19]:
# padding
padded_valid = pad_sequences(valid_text_times_dense, maxlen=max_len, dtype='float32', padding='post')
padded_valid.shape

(9262, 50)

In [20]:
# padding
padded_test = pad_sequences(test_text_times_dense, maxlen=max_len, dtype='float32', padding='post')
padded_test.shape

(11469, 50)

In [21]:
padded_train_varis = pad_sequences(train_varis, maxlen=max_len, dtype='float32', padding='post')
padded_train_varis.shape

(36551, 50)

In [22]:
padded_valid_varis = pad_sequences(valid_varis, maxlen=max_len, dtype='float32', padding='post')
padded_valid_varis.shape

(9262, 50)

In [23]:
padded_test_varis = pad_sequences(test_varis, maxlen=max_len, dtype='float32', padding='post')
padded_test_varis.shape

(11469, 50)

In [24]:
# # dump to pkl, update sepsis_removed_0.pkl
# pickle.dump([padded_train, padded_valid, padded_test, padded_train_varis, padded_valid_varis, padded_test_varis], open('Data/CLS/cls_text_times.pkl','wb'))

In [25]:
# dump to pkl, update sepsis_removed_0.pkl
pickle.dump([padded_train, padded_valid, padded_test, padded_train_varis, padded_valid_varis, padded_test_varis], open('Data/CLS/chrono/cls_text_times.pkl','wb'))